# Rank Classification using Tensorflow BERT on Amazon Review


<img src="https://djl-ai.s3.amazonaws.com/resources/images/amazon_review.png" width="500">
<center>Amazon Review example</center>

In [1]:
import $ivy.`ai.djl:api:0.17.0`
import $ivy.`ai.djl.tensorflow:tensorflow-model-zoo:0.17.0`
import $ivy.`ai.djl.tensorflow:tensorflow-engine:0.17.0`
import $ivy.`org.slf4j:slf4j-api:1.7.36`
import $ivy.`org.slf4j:slf4j-simple:1.7.36`

import $ivy.$                  

import $ivy.$                                              

import $ivy.$                                           

import $ivy.$                           

import $ivy.$                              

### Import java packages

In [2]:
import java.io.IOException
import java.nio.file.{Files, Paths}
import java.util

import ai.djl.modality.Classifications
import ai.djl.modality.nlp.DefaultVocabulary
import ai.djl.modality.nlp.bert.BertFullTokenizer
import ai.djl.modality.nlp.Vocabulary
import ai.djl.ndarray.NDList
import ai.djl.repository.zoo.Criteria
import ai.djl.training.util.{DownloadUtils, ProgressBar}
import ai.djl.translate.{Batchifier, Translator, TranslatorContext}
import ai.djl.util.ZipUtils

import java.io.IOException

import java.nio.file.{Files, Paths}

import java.util


import ai.djl.modality.Classifications

import ai.djl.modality.nlp.DefaultVocabulary

import ai.djl.modality.nlp.bert.BertFullTokenizer

import ai.djl.modality.nlp.Vocabulary

import ai.djl.ndarray.NDList

import ai.djl.repository.zoo.Criteria

import ai.djl.training.util.{DownloadUtils, ProgressBar}

import ai.djl.translate.{Batchifier, Translator, TranslatorContext}

import ai.djl.util.ZipUtils

## Prepare model files

In [3]:
val modelUrl = "https://resources.djl.ai/demo/tensorflow/amazon_review_rank_classification.zip"
DownloadUtils.download(
  modelUrl,
  "../build/tensorflow/amazon_review_rank_classification.zip",
  new ProgressBar
)
val zipFile = Paths.get("../build/tensorflow/amazon_review_rank_classification.zip")

val modelDir = Paths.get("../build/tensorflow/bert-rank")
if (Files.notExists(modelDir)) ZipUtils.unzip(Files.newInputStream(zipFile), modelDir)

modelDir: java.nio.file.Path = ../build/tensorflow/bert-rank

## Create Translator

In [4]:
class BertRankTranslator(var tokenizer: BertFullTokenizer, var length: Int)
    extends Translator[String, Classifications] {

  private val vocab: Vocabulary = tokenizer.getVocabulary
  private val ranks: util.List[String] = util.Arrays.asList("1", "2", "3", "4", "5")

  override def processInput(ctx: TranslatorContext, input: String): NDList = {
    val tokens = this.tokenizer.tokenize(input)
    val indices = new Array[Long](length)
    val mask = new Array[Long](length)
    val segmentIds = new Array[Long](length)
    val size = math.min(length, tokens.size)
    for (i <- 0 until size) {
      indices(i + 1) = vocab.getIndex(tokens.get(i))
    }
    util.Arrays.fill(mask, 0, size, 1)
    val m = ctx.getNDManager
    new NDList(m.create(indices), m.create(mask), m.create(segmentIds))
  }

  override def processOutput(ctx: TranslatorContext, list: NDList) =
    new Classifications(ranks, list.singletonOrThrow.softmax(0))
        
  override def getBatchifier: Batchifier = Batchifier.STACK
}

defined class BertRankTranslator

## Load tensorflow model

In [5]:
val modelDir = Paths.get("../build/tensorflow/bert-rank")
val vocabFile = modelDir.resolve("vocab.txt")
val vocabulary = DefaultVocabulary.builder
  .optMinFrequency(1)
  .addFromTextFile(vocabFile)
  .optUnknownToken("[UNK]")
  .build
val tokenizer = new BertFullTokenizer(vocabulary, true)
val maxTokenLength = 64 // cutoff tokens length

val translator = new BertRankTranslator(tokenizer, maxTokenLength)

val criteria = Criteria.builder
  .setTypes(classOf[String], classOf[Classifications])
  .optModelPath(modelDir)
  .optTranslator(translator)
  .optProgress(new ProgressBar)
  .build

val model = criteria.loadModel

val predictor = model.newPredictor

Loading:     100% |████████████████████████████████████████|


modelDir: java.nio.file.Path = ../build/tensorflow/bert-rank
vocabFile: java.nio.file.Path = ../build/tensorflow/bert-rank/vocab.txt
vocabulary: DefaultVocabulary = ai.djl.modality.nlp.DefaultVocabulary@3d12756d
tokenizer: BertFullTokenizer = ai.djl.modality.nlp.bert.BertFullTokenizer@5771529b
maxTokenLength: Int = 64
translator: BertRankTranslator = ammonite.$sess.cmd3$Helper$BertRankTranslator@6d31dea5
criteria: Criteria[String, Classifications] = Criteria:
	Application: UNDEFINED
	Input: class java.lang.String
	Output: class ai.djl.modality.Classifications
	ModelZoo: ai.djl.localmodelzoo

model: ai.djl.repository.zoo.ZooModel[String, Classifications] = ai.djl.repository.zoo.ZooModel@530a04bf
predictor: ai.djl.inference.Predictor[String, Classifications] = ai.djl.inference.Predictor@3815f5af

In [6]:
val review = "It works great, but it takes too long to update itself and slows the system"
predictor.predict(review)

review: String = "It works great, but it takes too long to update itself and slows the system"
res5_1: Classifications = [
	class: "4", probability: 0.21119
	class: "2", probability: 0.20799
	class: "5", probability: 0.19935
	class: "3", probability: 0.19762
	class: "1", probability: 0.18384
]

In [7]:
predictor.close()